# Project 1b


## Load data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as ply
train_data = pd.read_csv('train.csv').drop('Id', axis=1)

In [2]:
X_train = train_data.iloc[:, 1:].to_numpy()
y_train = train_data['y'].to_numpy()

## Build Transformer

In [45]:
def funky_transform(X):
    linear = X
    quadratic = X ** 2
    exponential = np.exp(X)
    cosine = np.cos(X)
    constant = np.ones(X.shape[0])
    constant = constant[:, np.newaxis]
    return np.hstack((linear, quadratic, exponential, cosine, constant))

## Regress

In [42]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import (GridSearchCV,
    cross_val_score, KFold)
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import Ridge

pipeline = make_pipeline(StandardScaler(),
                         FunctionTransformer(
                             funky_transform,
                             validate=True),
                         Ridge())
parameter_space = {
    'ridge__alpha': np.logspace(-2, 2, 10)
}
inner_cv = KFold(n_splits=7, shuffle=True)
outer_cv = KFold(n_splits=7, shuffle=True)
classifier = GridSearchCV(pipeline, parameter_space,
                               n_jobs=1, scoring='neg_mean_squared_error',
                               iid=True,
                               cv=inner_cv)
scores = cross_val_score(classifier, X_train, y_train.ravel(),
                            cv=outer_cv,
                            scoring='neg_mean_squared_error',
                            n_jobs=1)
print("Cross-validation score is {score:.3f},"
      " standart deviation is {err:.3f}"
      .format(score = np.sqrt(-scores).mean(), err = np.sqrt(-scores).std()))

classifier = classifier.fit(X_train, y_train.ravel())
coefficients = classifier.best_estimator_['ridge'].coef_

Cross-validation score is 4.922, standart deviation is 0.552


## Log to solution.csv file

In [44]:
df = pd.DataFrame(coefficients)
df.to_csv('solution.csv', index=False, header=False)